# Baseline 설명 추가 ver.

## 파일 입력 받기

파일 입력은 pandas라이브러리의 `read_csv()`함수를 사용하여 할 수 있습니다.

In [ ]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl

train = pd.read_csv('/kaggle/input/kakr-4th-competition/train.csv')
test = pd.read_csv('/kaggle/input/kakr-4th-competition/test.csv')
sample_submission = pd.read_csv('/kaggle/input/kakr-4th-competition/sample_submission.csv')

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
train.describe()

In [ ]:
fig, ax=plt.subplots(1,1, figsize=(12,5))
ax.hist(train['age'],width=2, edgecolor='black')
ax.set_title('Age distribution',fontweight='bold')
plt.show()

In [ ]:
## 밀도함수
#sns.displot(data=train, x='age')
#train['age']=train['age'].map(lambda x: int(x))
sns.kdeplot(train['age'])

In [ ]:
import missingno as msno
msno.matrix(train) ##null 데이터 확인

In [ ]:
train['workclass'].value_counts()

In [ ]:

fig, ax= plt.subplots(1,1,figsize=(12,5))
sns.countplot(data=train, x='workclass', ax=ax)
plt.show()

In [ ]:
fig, ax= plt.subplots(1,1,figsize=(12,5))
sns.countplot(data=train, x='race', ax=ax, hue='sex')
plt.show()

## EDA

데이터를 이리저리 살피는 과정이 있어야 합니다.

- 어떤 문제를 해결하려고 하고 있는가?
- 어떤 종류의 데이터가 있고, 이를 어떻게 처리할까?
- 데이터에 누락값이 무엇이고, 이를 어떻게 처리할까?
- 특이값은 어디에 있으며 이를 어떻게 처리할까?
- 데이터를 최대한 활용하기 위해 어떻게 새로운 피처를 만들까?

위 질문에 대해서 살펴보기 위해 다양한 시각화를 진행할 수 있습니다.

결측치의 경우, 다음 시각화를 사용할 수 있습니다.

In [ ]:
import missingno as msno
msno.matrix(train)

## 모델에 적합한 input 형태로 만들기

features(X)와 target(y)으로 나누어 모델에 넣어주어야 합니다.

그렇기에 우리가 예측해야하는 `income` feature는 따로 빼겠습니다.

In [ ]:
train['income']

In [ ]:
# income 을 0,1로 바꿔주기
train['income'] = (train['income'] != '<=50K').astype(int)
#target


In [ ]:
# 피벗테이블 만들기, 인종으로 먼저 묶고 성별로 묶은 다음에 평균을 낸것
#train.groupby(['race', 'sex']).mean()

#배경색을 값에 따라 바꿔주는 것
train.groupby(['race', 'sex'])[['income']].mean().style.background_gradient(cmap='Purples')

In [ ]:
pd.pivot_table(train, columns='sex', index='race', values='income',aggfunc='mean')

In [ ]:
train

In [ ]:
# 분포 확인하기
fig, ax=plt.subplots(1,1, figsize=(10,6))
#sns.boxplot(data=train, x='race', y='age', hue='sex')
sns.violinplot(data=train, x='race', y='age')
#swarmplot

In [ ]:
train.corr() ## 상관계수(숫자형 데이터)


In [ ]:
#train.corr().style.background_gradient()
train.corr().style.background_gradient(cmap='coolwarm')

In [ ]:
corr=train.corr()
sns.heatmap(corr)

In [ ]:
#sns.heatmap(corr, square=True, linecolor='white', linewidth='0.1', cmap='coolwarm')
sns.heatmap(corr, square=True, linecolor='white', linewidth='0.1', cmap='coolwarm',
           vmax=1.0, vmin=-1.0)

In [ ]:
target=train['income']
target

In [ ]:
train.drop(['income'], axis=1, inplace=True)

In [ ]:
train.head()

categorical feature의 경우에는 일반적인 모델에서 바로 사용할 수 없기 때문에 변환(encoding)이 필요합니다.

여기서도 다양한 Encoding 방법이 있습니다. 

- Label Encoding
- One-Hot Encoding
- Mean Encoding

등이 존재하고, category_encoders 라이브러리를 사용하여 Encoding을 진행할 수 있습니다.

In [ ]:
from category_encoders.ordinal import OrdinalEncoder
LE_encoder = OrdinalEncoder(list(train.columns))# 모든 컬럼에 대해 인코딩 진행
train_le = LE_encoder.fit_transform(train, target)
test_le = LE_encoder.transform(test)

그 외에도 다양한 feature engineering 또는 feature selection 을 할 수 있지만 여기서는 생략합니다.

## 과적합 방지를 위한 Test / Validation 분리

원래는 데이터의 과적합을 방직하기 위해 test dataset 이전에 validation dataset을 만들어 train dataset으로 훈련한 결과를 확인합니다.

그리고 이런 validation에도 여러가지 방법이 존재합니다.
특히 캐글에서는 Cross-Validation을 많이 사용하는데 

- KFold
- Stratified KFold

등 여러가지 방법이 있습니다.

이 커널에서는 따로 다루지 않습니다.

## 모델 훈련 및 예측

scikit-learn에서 제공하는 대부분의 모델은 train, target을 입력으로 받아 훈련을 진행합니다.

하이퍼파라미터의 차이가 있지만 모델 종류에 따라 커스텀해도 되겠죠?

- Decision Tree
- KNN
- Linear Classification
- Logistic Regression
- Lasso
- Ridge
- Random Forest
- XGBoost
- Perceptron

이런 모델은 sklearn 또는 cuml, pycaret 등의 라이브러리에 구현되어 있습니다.
찾아보시면서 하시면 좋습니다.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt_clf = DecisionTreeClassifier()
dt_clf.fit(train_le, target)


## 결과 제출

예비 제출 파일을 사용하여 다음과 같이 제출할 수 있습니다.

제출 파일은 csv 파일로 제출해야하며, `to_csv` 메서드를 사용하여 캐글 서버 내에 csv 파일을 만들 수 있습니다.

In [ ]:
dt_clf.predict(test_le)

In [ ]:
sample_submission['prediction'] = dt_clf.predict(test_le).astype(int)
sample_submission.to_csv('submission.csv', index=False)